In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

# Load the extended dataset
data = pd.read_csv('expanded_vsd_dataset.csv')

# Cleaning Dataset
## Missing Values
data = data.dropna()

## Categorical Columns
categorical_columns = data.select_dtypes(include=['object']).columns
data[categorical_columns] = data[categorical_columns].fillna('Unknown')

## Numeric Columns
numeric_columns = data.select_dtypes(include=['number']).columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# Define features and labels for heart disease prediction
X = data.drop(columns=['Has VSD', 'Other Condition', 'Severity'])
y_vsd = data['Has VSD']                # Target for VSD detection
y_condition = data['Other Condition']  # Target for other conditions
y_severity = data['Severity']          # Target for VSD severity

# One-hot encoding for categorical features
X = pd.get_dummies(X, columns=['Cholesterol'], drop_first=True)

# Split data into training and testing sets
X_train, X_test, y_vsd_train, y_vsd_test = train_test_split(X, y_vsd, test_size=0.2, random_state=42)
_, _, y_condition_train, y_condition_test = train_test_split(X, y_condition, test_size=0.2, random_state=42)
_, _, y_severity_train, y_severity_test = train_test_split(X, y_severity, test_size=0.2, random_state=42)

# Train Models
## VSD Prediction
vsd_model = DecisionTreeClassifier(random_state=42)
vsd_model.fit(X_train, y_vsd_train)

## Condition Prediction
condition_model = RandomForestClassifier(random_state=42, n_estimators=100)
condition_model.fit(X_train, y_condition_train)

## Severity Prediction
severity_model = RandomForestClassifier(random_state=42, n_estimators=100)
severity_model.fit(X_train, y_severity_train)

# Example patient data
patient_data = [[170, 50.93, 110.15, 2.76, 99.59, 51.79, 76, 0, 0, 90, 75, 1, 1]]
patient_df = pd.DataFrame(patient_data, columns=X.columns)  # Create a DataFrame with matching column names

# Predict VSD
vsd_prediction = vsd_model.predict(patient_df)[0]
vsd_result = "Has VSD" if vsd_prediction == 1 else "VSD absent"
print(f"VSD Prediction: {vsd_result}")

# Predict Condition (if required)
condition_prediction = condition_model.predict(patient_df)[0]
print(f"Condition Prediction: {condition_prediction}")

# Predict Severity
severity_prediction = severity_model.predict(patient_df)[0]
print(f"Severity Prediction: {severity_prediction}")

# Accuracy Evaluation
vsd_accuracy = accuracy_score(y_vsd_test, vsd_model.predict(X_test))
severity_accuracy = accuracy_score(y_severity_test, severity_model.predict(X_test))

print(f"VSD Prediction Accuracy: {vsd_accuracy:.2f}")
print(f"Severity Prediction Accuracy: {severity_accuracy:.2f}")

# Save models
pickle.dump(vsd_model, open("vsd_model.pkl", "wb"))
pickle.dump(severity_model, open("severity_model.pkl", "wb"))
pickle.dump(condition_model, open("condition_model.pkl", "wb"))

VSD Prediction: Has VSD
Condition Prediction: Atrial Septal Defect
Severity Prediction: Mild
VSD Prediction Accuracy: 1.00
Severity Prediction Accuracy: 1.00


In [3]:
# Treatment Recommendation Logic
def recommend_treatment(vsd_present, severity, condition, cyanosis):
    if vsd_present == 0:
        return "No VSD detected. No treatment required."
    
    recommendations = []
    if severity == "Mild":
        recommendations.append("Monitor periodically and prescribe medication if needed.")
        if condition == "None":
            recommendations.append("Healthy lifestyle and regular check-ups.")
        else:
            recommendations.append(f"Address secondary condition: {condition}.")
    elif severity == "Moderate":
        recommendations.append("Further investigation recommended. Consider non-invasive procedures.")
        if condition != "None":
            recommendations.append(f"Manage secondary condition: {condition}.")
    elif severity == "Severe":
        recommendations.append("Surgical intervention or advanced treatment recommended.")
        if cyanosis == 1:
            recommendations.append("Immediate action needed due to cyanosis.")

    return " ".join(recommendations)




#Testing and Evaluating models
patient_data = [[170, 50.93, 110.15, 2.76, 99.59, 51.79, 76, 0, 0, 90, 75, 1, 1]] 
patient_df = pd.DataFrame(patient_data, columns=X.columns)

# Predict VSD
vsd_prediction = vsd_model.predict(patient_df)[0]
vsd_result = "Has VSD" if vsd_prediction == 1 else "VSD absent"

# Predict Severity
severity_prediction = severity_model.predict(patient_df)[0]

# Predict Other Condition
condition_prediction = condition_model.predict(patient_df)[0]

# Get Treatment Recommendation
cyanosis = patient_df['Cyanosis'].values[0]  
treatment = recommend_treatment(vsd_prediction, severity_prediction, condition_prediction, cyanosis)

# Output Results
print(f"VSD Prediction: {vsd_result}")
print(f"Severity: {severity_prediction}")
print(f"Other Condition: {condition_prediction}")
print(f"Treatment Recommendation: {treatment}")


VSD Prediction: Has VSD
Severity: Mild
Other Condition: Atrial Septal Defect
Treatment Recommendation: Monitor periodically and prescribe medication if needed. Address secondary condition: Atrial Septal Defect.


In [3]:
# Prognosis and Follow-Up Recommendation
def personalized_recommendations(severity, age, gender, weight, height, condition, cyanosis):
    recommendations = []

    
    # Prognosis based on severity
    if severity == "Mild":
        prognosis = "Low Risk"
        recommendations.append("Maintain a healthy lifestyle and schedule yearly check-ups.")
    elif severity == "Moderate":
        prognosis = "Moderate Risk"
        recommendations.append("Follow up every 6 months. Consider reducing physical exertion.")
    elif severity == "Severe":
        prognosis = "High Risk"
        recommendations.append("Immediate medical intervention required. Regular follow-ups every 3 months.")


    
    # Personalized recommendations based on patient demographics
    bmi = weight / ((height / 100) ** 2)
    if bmi < 18.5:
        recommendations.append("BMI indicates underweight. Consult a nutritionist to improve diet.")
    elif bmi > 24.9:
        recommendations.append("BMI indicates overweight. Encourage a balanced diet and exercise.")

    # Gender-specific advice (e.g., females with heart conditions)
    if gender == 1:  # Assuming 1 = Female
        recommendations.append("Female patients with VSD should monitor hormonal changes closely.")


    
    # Address secondary conditions
    if condition != "None":
        recommendations.append(f"Monitor and manage secondary condition: {condition}.")

    # Cyanosis alert
    if cyanosis == 1:
        recommendations.append("Cyanosis detected. Seek urgent medical attention.")

    return prognosis, " ".join(recommendations)



# Testing and Evaluating Models
patient_data = [[170, 50.93, 110.15, 2.76, 99.59, 51.79, 76, 0, 0, 90, 75, 1, 1]]  # Replace with actual patient data
patient_df = pd.DataFrame(patient_data, columns=X.columns)

# Predict VSD
vsd_prediction = vsd_model.predict(patient_df)[0]
vsd_result = "Has VSD" if vsd_prediction == 1 else "VSD absent"

# Predict Severity
severity_prediction = severity_model.predict(patient_df)[0]

# Predict Other Condition
condition_prediction = condition_model.predict(patient_df)[0]



# Get Personalized Prognosis and Recommendations
age = patient_df['Age'].values[0]
gender = patient_df['Gender'].values[0]
weight = patient_df['Weight (kg)'].values[0]
height = patient_df['Height (cm)'].values[0]
cyanosis = patient_df['Cyanosis'].values[0]

prognosis, follow_up_recommendations = personalized_recommendations(
    severity=severity_prediction,
    age=age,
    gender=gender,
    weight=weight,
    height=height,
    condition=condition_prediction,
    cyanosis=cyanosis
)

# Output Results
print(f"VSD Prediction: {vsd_result}")
print(f"Severity: {severity_prediction}")
print(f"Other Condition: {condition_prediction}")
print(f"Prognosis: {prognosis}")
print(f"Follow-Up Recommendations: {follow_up_recommendations}")
print("hello world")

VSD Prediction: Has VSD
Severity: Mild
Other Condition: Atrial Septal Defect
Prognosis: Low Risk
Follow-Up Recommendations: Maintain a healthy lifestyle and schedule yearly check-ups. BMI indicates overweight. Encourage a balanced diet and exercise. Monitor and manage secondary condition: Atrial Septal Defect.
hello world
